# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [3]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../output_data/cities.csv")

# Display sample data
city_data_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,sur,22.5667,59.5289,25.44,60,0,4.32,OM,1731637688
1,adamstown,-25.0660,-130.1015,23.31,87,80,3.93,PN,1731637689
2,kirakira,-10.4544,161.9205,29.61,71,49,4.26,SB,1731637691
3,dolbeau-mistassini,48.8763,-72.2120,-2.74,68,100,2.27,CA,1731637692
4,aasiaat,68.7098,-52.8699,-16.41,99,93,4.82,GL,1731637693


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [27]:
%%capture --no-display

# Configure the map plot
world_map = city_data_df.hvplot.points(
    'Lng',
    'Lat',  
    geo = True,
    color='City',
    size='Humidity',
    tiles="OSM", 
    alpha=0.7,
    frame_width = 810,
    frame_height = 520,
)

# Display the map
world_map

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [29]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_weather = city_data_df[
    (city_data_df['Max Temp'] < 27) &
    (city_data_df['Max Temp'] > 21) &
    (city_data_df['Wind Speed'] < 4.5) &
    (city_data_df['Cloudiness'] == 0)
]

# Drop any rows with null values
new_ideal_weather = ideal_weather.dropna()

# Display sample data
new_ideal_weather

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,sur,22.5667,59.5289,25.44,60,0,4.32,OM,1731637688
78,cabo san lucas,22.8909,-109.9124,25.42,59,0,4.12,MX,1731637609
129,ringas,27.3500,75.5667,22.73,32,0,1.08,IN,1731637841
222,port isabel,26.0734,-97.2086,21.02,73,0,3.09,US,1731637955
243,bela,25.9333,81.9833,23.17,36,0,1.97,IN,1731637979
313,tura,25.5198,90.2201,24.11,65,0,0.50,IN,1731638062
348,kot diji,27.3427,68.7083,25.23,46,0,2.29,PK,1731638102
399,katsina,12.9894,7.6006,21.84,20,0,4.20,NG,1731638164
414,jianshui,23.6363,102.8264,21.09,53,0,1.80,CN,1731638181
461,nongpoh,25.9000,91.8833,22.00,68,0,1.24,IN,1731638236


### Step 3: Create a new DataFrame called `hotel_df`.

In [36]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = new_ideal_weather[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = None

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
0,sur,OM,22.5667,59.5289,60,None
78,cabo san lucas,MX,22.8909,-109.9124,59,None
129,ringas,IN,27.3500,75.5667,32,None
222,port isabel,US,26.0734,-97.2086,73,None
243,bela,IN,25.9333,81.9833,36,None
313,tura,IN,25.5198,90.2201,65,None
348,kot diji,PK,27.3427,68.7083,46,None
399,katsina,NG,12.9894,7.6006,20,None
414,jianshui,CN,23.6363,102.8264,53,None
461,nongpoh,IN,25.9000,91.8833,68,None


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [74]:
# Set parameters to search for a hotel
radius = 10000
limit = 1
filters = f"circle:{lon},{lat},{radius}"
bias = f"proximity:{lon},{lat}"

params = {
    "categories": "accommodation.hotel",
    "filter":filters,
    "limit":limit,
    "bias":bias,
    "apiKey":geoapify_key    
}

# Set base URL
base_url = "https://api.geoapify.com/v2/places"

In [78]:
requests.get(base_url, params=params)

<Response [200]>

In [79]:
# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row['Lat']
    lon = row['Lng']

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lon},{lat},{radius}"
    params["bias"] = f"proximity:{lon},{lat}"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary, Convert the API response to JSON format
    response = requests.get(base_url, params=params)
    name_address = response.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
sur - nearest hotel: Sur Hotel
cabo san lucas - nearest hotel: Comfort Rooms
ringas - nearest hotel: No hotel found
port isabel - nearest hotel: Queen Isabel Inn
bela - nearest hotel: Hotel City Palace
tura - nearest hotel: Hotel Polo Orchid
kot diji - nearest hotel: No hotel found
katsina - nearest hotel: Fadama view Hotel
jianshui - nearest hotel: 阳阳大酒店
nongpoh - nearest hotel: No hotel found
bitkine - nearest hotel: No hotel found
sultanpur - nearest hotel: Raj Hotel
bachhrawan - nearest hotel: No hotel found
bahia de kino - nearest hotel: Hotel Hacienda


,City,Country,Lat,Lng,Humidity,Hotel Name
0,sur,OM,22.5667,59.5289,60,Sur Hotel
78,cabo san lucas,MX,22.8909,-109.9124,59,Comfort Rooms
129,ringas,IN,27.3500,75.5667,32,No hotel found
222,port isabel,US,26.0734,-97.2086,73,Queen Isabel Inn
243,bela,IN,25.9333,81.9833,36,Hotel City Palace
313,tura,IN,25.5198,90.2201,65,Hotel Polo Orchid
348,kot diji,PK,27.3427,68.7083,46,No hotel found
399,katsina,NG,12.9894,7.6006,20,Fadama view Hotel
414,jianshui,CN,23.6363,102.8264,53,阳阳大酒店
461,nongpoh,IN,25.9000,91.8833,68,No hotel found


In [80]:
response.json()

{'type': 'FeatureCollection',
 'features': [{'type': 'Feature',
   'properties': {'name': 'Hotel Hacienda',
    'country': 'Mexico',
    'country_code': 'mx',
    'state': 'Sonora',
    'county': 'Hermosillo',
    'city': 'Bahía Kino',
    'district': 'Bahía Kino',
    'street': 'Calle Manzanillo',
    'lon': -111.9422538,
    'lat': 28.825420200000003,
    'state_code': 'SON',
    'formatted': 'Hotel Hacienda, Calle Manzanillo, Bahía Kino, SON, Mexico',
    'address_line1': 'Hotel Hacienda',
    'address_line2': 'Calle Manzanillo, Bahía Kino, SON, Mexico',
    'categories': ['access_limited',
     'access_limited.customers',
     'accommodation',
     'accommodation.hotel',
     'internet_access',
     'internet_access.free',
     'no_fee',
     'no_fee.no',
     'parking',
     'parking.cars'],
    'details': ['details.contact', 'details.facilities', 'details.parking'],
    'datasource': {'sourcename': 'openstreetmap',
     'attribution': '© OpenStreetMap contributors',
     'license

### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [88]:
%%capture --no-display

# Configure the map plot
hotels_map = hotel_df.hvplot.points(
    'Lng',
    'Lat',  
    geo = True,
    color='City',
    size='Humidity',
    tiles="OSM", 
    alpha=0.7,
    frame_width = 900,
    frame_height = 700,
    hover_cols=['City','Humidity', 'Country', 'Hotel Name']
)

# map the plot
hotels_map


:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Country,Hotel Name)